In [1]:
pip install evalml

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requirements-parser>=0.2.0 in c:\users\rohit.pratapwar\anaconda3\lib\site-packages (from evalml) (0.2.0)
  Using cached pandas-1.1.5-cp37-cp37m-win_amd64.whl (8.7 MB)



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
automl 2.9.9 requires dill<0.3,>=0.2.5, but you have dill 0.3.3 which is incompatible.
automl 2.9.9 requires lightgbm<2.1,>=2.0.11, but you have lightgbm 3.0.0 which is incompatible.
automl 2.9.9 requires pandas<1.0,>=0.18.0, but you have pandas 1.1.5 which is incompatible.


  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.3
    Uninstalling pandas-1.2.3:
      Successfully uninstalled pandas-1.2.3


In [2]:
import evalml
from evalml import AutoMLSearch
from evalml.objectives import FraudCost

In [3]:
fraud_objectives = FraudCost(retry_percentage=0.5,interchange_fee=0.02,fraud_payout_percentage=.75,amount_col='amount')

In [4]:
features,target = evalml.demos.load_fraud(n_rows=10000)

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 10000
Targets
False    85.30%
True     14.70%
Name: fraud, dtype: object


In [5]:
cols_to_drop = ['datetime', 'expiration_date', 'country', 'region', 'provider']

In [6]:
for col in cols_to_drop:
    features.pop(col)

In [7]:
features_train,features_test,target_train,target_test = evalml.preprocessing.split_data(features,target,problem_type='binary',test_size=0.2)

In [8]:
print(features.types)

                 Physical Type Logical Type Semantic Tag(s)
Data Column                                                
card_id                  Int64      Integer     ['numeric']
store_id                 Int64      Integer     ['numeric']
amount                   Int64      Integer     ['numeric']
currency              category  Categorical    ['category']
customer_present       boolean      Boolean              []
lat                    float64       Double     ['numeric']
lng                    float64       Double     ['numeric']


In [9]:
features.head()

Data Column,card_id,store_id,amount,currency,customer_present,lat,lng
Physical Type,Int64,Int64,Int64,category,boolean,float64,float64
Logical Type,Integer,Integer,Integer,Categorical,Boolean,Double,Double
Semantic Tag(s),['numeric'],['numeric'],['numeric'],['category'],[],['numeric'],['numeric']
id,,,,,,,
0,32261,8516,24900,CUC,True,38.58894,-89.99038
1,16434,8516,15789,MYR,False,38.58894,-89.99038
2,23468,8516,1883,AUD,False,38.58894,-89.99038
3,14364,8516,82120,KRW,True,38.58894,-89.99038
4,29407,8516,25745,MUR,True,38.58894,-89.99038


In [10]:
automl = AutoMLSearch(X_train=features_train,y_train=target_train,problem_type='binary',objective=fraud_objectives,
                     additional_objectives=['auc','f1','precision'],
                     max_batches = 1, optimize_thresholds=True)

In [11]:
automl.search()

Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Cost. 
Lower score is better.

Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: linear_model, xgboost, extra_trees, catboost, random_forest, decision_tree, lightgbm



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/9) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.040
Batch 1: (2/9) Logistic Regression Classifier w/ Imp... Elapsed:00:01
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.040
Batch 1: (3/9) Random Forest Classifier w/ Imputer +... Elapsed:00:10
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.002
Batch 1: (4/9) XGBoost Classifier w/ Imputer + One H... Elapsed:00:15
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.028
High coefficient of variation (cv >= 0.2) within cross validation scores. XGBoost Classifier w/ Imputer + One Hot Encoder may not perform as estimated on unseen data.
Batch 1: (5/9) CatBoost Classifier w/ Imputer           Elapsed:00:18
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.002
Batch 1: (6/9) Elastic Net Classifier w/ Imputer + O... Elapsed:00:21
	Starting cross va

In [12]:
automl.rankings

,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,2,Random Forest Classifier w/ Imputer + One Hot ...,0.002104,0.002285,94.801712,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,4,CatBoost Classifier w/ Imputer,0.002104,0.002285,94.801712,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,5,Elastic Net Classifier w/ Imputer + One Hot En...,0.002104,0.002285,94.801712,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,6,Extra Trees Classifier w/ Imputer + One Hot En...,0.016676,0.046003,58.793775,True,{'Imputer': {'categorical_impute_strategy': 'm...
4,8,Decision Tree Classifier w/ Imputer + One Hot ...,0.017532,0.046003,56.679694,True,{'Imputer': {'categorical_impute_strategy': 'm...
5,3,XGBoost Classifier w/ Imputer + One Hot Encoder,0.028438,0.046003,29.732253,True,{'Imputer': {'categorical_impute_strategy': 'm...
6,7,LightGBM Classifier w/ Imputer + One Hot Encoder,0.038657,0.046003,4.482635,False,{'Imputer': {'categorical_impute_strategy': 'm...
7,1,Logistic Regression Classifier w/ Imputer + On...,0.040264,0.046003,0.511060,False,{'Imputer': {'categorical_impute_strategy': 'm...
8,0,Mode Baseline Binary Classification Pipeline,0.040471,0.046003,0.000000,False,{'Baseline Classifier': {'strategy': 'mode'}}


In [13]:
best_pipeline = automl.best_pipeline

In [15]:
automl.describe_pipeline(automl.rankings.iloc[1]['id'])

**********************************
* CatBoost Classifier w/ Imputer *
**********************************

Problem Type: binary
Model Family: CatBoost

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. CatBoost Classifier
	 * n_estimators : 10
	 * eta : 0.03
	 * max_depth : 6
	 * bootstrap_type : None
	 * silent : True
	 * allow_writing_files : False

Training
Training for binary problems.
Objective to optimize binary classification pipeline thresholds for: <evalml.objectives.fraud_cost.FraudCost object at 0x00000217F978C848>
Total training time (including CV): 2.8 seconds

Cross Validation
----------------
             Fraud Cost   AUC    F1  Precision # Training # Validation
0                 0.002 0.859 0.256      0.147   4266.000     2667.000
1                 0.002 0.844 0.256      0.147   4266.000     2667.000
2                 0.002 0.841 0.256      0.147 

In [16]:
best_pipeline.score(features_test,target_test,objectives=['auc',fraud_objectives])

OrderedDict([('AUC', 0.8680626998747917),
             ('Fraud Cost', 0.0014668699644176933)])

In [17]:
automl_auc = AutoMLSearch(X_train=features_train,y_train=target_train,problem_type='binary',
                         objective='auc',
                         additional_objectives=['f1','precision'],
                         max_batches=1,
                         optimize_thresholds=True)

In [18]:
automl_auc.search()

Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. 
Greater score is better.

Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: linear_model, xgboost, extra_trees, catboost, random_forest, decision_tree, lightgbm



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/9) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.500
Batch 1: (2/9) Logistic Regression Classifier w/ Imp... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.838
Batch 1: (3/9) Random Forest Classifier w/ Imputer +... Elapsed:00:06
	Starting cross validation
	Finished cross validation - mean AUC: 0.865
Batch 1: (4/9) XGBoost Classifier w/ Imputer + One H... Elapsed:00:09
	Starting cross validation
	Finished cross validation - mean AUC: 0.871
Batch 1: (5/9) CatBoost Classifier w/ Imputer           Elapsed:00:11
	Starting cross validation
	Finished cross validation - mean AUC: 0.847
Batch 1: (6/9) Elastic Net Classifier w/ Imputer + O... Elapsed:00:12
	Starting cross validation
	Finished cross validation - mean AUC: 0.500
Batch 1: (7/9) Extra Trees Classifier w/ Imputer + O... Elapsed:00:14
	Starting cross validation
	Finished cross validation - mean AUC: 0.845
Batch 

In [19]:
automl_auc.rankings

,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,3,XGBoost Classifier w/ Imputer + One Hot Encoder,0.871193,0.875069,74.238628,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,7,LightGBM Classifier w/ Imputer + One Hot Encoder,0.869615,0.877111,73.922916,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,2,Random Forest Classifier w/ Imputer + One Hot ...,0.864655,0.867795,72.931006,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,4,CatBoost Classifier w/ Imputer,0.847067,0.852940,69.413486,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,6,Extra Trees Classifier w/ Imputer + One Hot En...,0.845074,0.858111,69.014730,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,1,Logistic Regression Classifier w/ Imputer + On...,0.838197,0.852107,67.639415,False,{'Imputer': {'categorical_impute_strategy': 'm...
6,8,Decision Tree Classifier w/ Imputer + One Hot ...,0.725530,0.839576,45.105954,True,{'Imputer': {'categorical_impute_strategy': 'm...
7,0,Mode Baseline Binary Classification Pipeline,0.500000,0.500000,0.000000,False,{'Baseline Classifier': {'strategy': 'mode'}}
8,5,Elastic Net Classifier w/ Imputer + One Hot En...,0.500000,0.500000,0.000000,False,{'Imputer': {'categorical_impute_strategy': 'm...


In [20]:
best_pipeline_auc = automl_auc.best_pipeline

In [21]:
best_pipeline_auc.score(features_test,target_test,objectives=['auc',fraud_objectives])

OrderedDict([('AUC', 0.8693586461548277),
             ('Fraud Cost', 0.029556847741088976)])

In [22]:
best_pipeline.score(features_test,target_test,objectives=['auc',fraud_objectives])

OrderedDict([('AUC', 0.8680626998747917),
             ('Fraud Cost', 0.0014668699644176933)])